In [1]:
import chemdataextractor

In [2]:
from chemdataextractor import Document

In [3]:
from chemdataextractor.doc import Sentence

In [4]:
import pubchempy as pcp

In [5]:
import pandas as pd

In [6]:
import re

In [7]:
from tika import parser
raw1 = parser.from_file('ovarian_cancer.pdf')
raw2 = parser.from_file('ovarian_cancer2.pdf')
raw3 = parser.from_file('ovarian_cancer3.pdf')
raw4 = parser.from_file('ovarian_cancer4.pdf')
raw5 = parser.from_file('ovarian_cancer5.pdf')
raw6 = parser.from_file('ovarian_cancer6.pdf')
raw7 = parser.from_file('OvarianPaper.pdf')

In [8]:
doc=raw1['content']+raw2['content']+raw3['content']+raw4['content']+raw5['content']+raw6['content']+raw7['content']

In [9]:
df=pd.DataFrame(doc.split("."))
df.columns=['Sentence #']

In [10]:
df.head()

,Sentence #
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,"Armstrong, MD\n\n2\n\nAbstract\nEpithelial ov..."
2,Screening strategies using ultrasound and the...
3,Women\n\nwho have inherited a deleterious mut...
4,Other less common and less well-defined genet...


In [11]:
df.shape

(7268, 1)

In [12]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
import nltk
stopwords=nltk.corpus.stopwords.words('english')

In [14]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer=WordNetLemmatizer()

In [15]:
def clean(text):
    #clean_text="".join([i for i in text if i not in string.punctuation])
    clean_text=re.sub("\n"," ",text)
    clean_text="".join([wordnet_lemmatizer.lemmatize(i) for i in clean_text])
    clean_text="".join(i if i not in string.punctuation else "" for i in clean_text)
    return clean_text.strip()

In [16]:
df['clean']=df["Sentence #"].apply(clean)

In [17]:
df.head()

,Sentence #,clean
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Recent progress in the diagnosis and treatment...
1,"Armstrong, MD\n\n2\n\nAbstract\nEpithelial ov...",Armstrong MD 2 Abstract Epithelial ovarian c...
2,Screening strategies using ultrasound and the...,Screening strategies using ultrasound and the ...
3,Women\n\nwho have inherited a deleterious mut...,Women who have inherited a deleterious mutati...
4,Other less common and less well-defined genet...,Other less common and less welldefined genetic...


In [18]:
dictionary=pd.read_excel("OvarianCancer.xlsx")

def get_cid(arr):
    for i in arr:
            val=pcp.get_cids(i,"name")
            if val!=[]:
                if val[0] in list(df.CID):
                    return str(val[0])
    return(None)

In [104]:
def tokens_n_tags(text):
    l=[tn for tn in text.split() if tn not in stopwords]
    m1=pd.DataFrame((rec["names"],get_cid(rec["names"])) for rec in Document(text).records.serialize() if "names" in rec.keys())
    m1.dropna(inplace=True)
    if not m1.empty:
        m1.columns=["Name","CID"]
        m1=m1.apply(lambda x: x.explode() if x.name in ["Name"] else x)
        list_m1=list(m1["Name"])
    else:
        return l,["O" for i in range(len(l))]
    tokens,tags=[],[]
    i=0
    while i<len(l):
        flag=0
        tkn=l[i]
        for t in list_m1:
            if tkn in t:
                ln=len(t.split())
                tokens.append(t)
                if tkn.isupper():
                    tags.append("Compund-Abv")
                else:
                    tags.append("Compund-Name")
                i=i+ln
                flag=1
                break    
        if flag==0:
            tokens.append(tkn)
            tags.append("O")
            i=i+1
    return tokens,tags

In [101]:
text=df.loc[441,"clean"]

In [102]:
text

'94  Gemcitabine has been studied as a single agent in  platinumresistant disease with response rates of  11 to 17'

In [99]:
tokens_n_tags(text)

0 94
1 Gemcitabine
2 Gemcitabine 1
2 studied
3 single
4 agent
5 platinumresistant
6 disease
7 response
8 rates
9 11
10 17


(['94',
  'Gemcitabine',
  'studied',
  'single',
  'agent',
  'platinumresistant',
  'disease',
  'response',
  'rates',
  '11',
  '17'],
 ['O', 'Compund-Name', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [105]:
tagntkn=df["clean"].apply(lambda x:tokens_n_tags(x))

In [106]:
df["Word"]=[i[0] for i in tagntkn]
df["Tags"]=[i[1] for i in tagntkn]

In [107]:
df.head()

,Sentence #,clean,Word,Tags
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Recent progress in the diagnosis and treatment...,"[Recent, progress, diagnosis, treatment, ovari...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"Armstrong, MD\n\n2\n\nAbstract\nEpithelial ov...",Armstrong MD 2 Abstract Epithelial ovarian c...,"[Armstrong, MD, 2, Abstract, Epithelial, ovari...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Screening strategies using ultrasound and the...,Screening strategies using ultrasound and the ...,"[Screening, strategies, using, ultrasound, can...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,Women\n\nwho have inherited a deleterious mut...,Women who have inherited a deleterious mutati...,"[Women, inherited, deleterious, mutation, BRCA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,Other less common and less well-defined genet...,Other less common and less welldefined genetic...,"[Other, less, common, less, welldefined, genet...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [108]:
len(df)

7268

In [ ]:
for i in range(len(df)):
    if len(df.Tags[i])==df.Tags[i].count("O"):
        df.drop(i,inplace=True)

In [ ]:
len(df)

In [ ]:
df.reset_index(inplace=True)

In [109]:
df["Sentence #"]=df.index+1

In [110]:
df.head()

,Sentence #,clean,Word,Tags
0,1,Recent progress in the diagnosis and treatment...,"[Recent, progress, diagnosis, treatment, ovari...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,2,Armstrong MD 2 Abstract Epithelial ovarian c...,"[Armstrong, MD, 2, Abstract, Epithelial, ovari...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,3,Screening strategies using ultrasound and the ...,"[Screening, strategies, using, ultrasound, can...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,4,Women who have inherited a deleterious mutati...,"[Women, inherited, deleterious, mutation, BRCA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,5,Other less common and less welldefined genetic...,"[Other, less, common, less, welldefined, genet...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [111]:
df.drop("clean",axis=1,inplace=True)

In [112]:
df=df.apply(lambda x: x.explode() if x.name in ["Word","Tags"] else x)

In [113]:
df.head()

,Sentence #,Word,Tags
0,1,Recent,O
0,1,progress,O
0,1,diagnosis,O
0,1,treatment,O
0,1,ovarian,O


In [114]:
df["POS"]=df["Word"].apply(lambda x:Sentence(str(x)).pos_tagged_tokens[0][1])

In [115]:
df.head()

,Sentence #,Word,Tags,POS
0,1,Recent,O,JJ
0,1,progress,O,NN
0,1,diagnosis,O,NN
0,1,treatment,O,NN
0,1,ovarian,O,JJ


In [116]:
df.to_excel("FinalDataset4.xlsx")